In [2]:
import pandas as pd

import re
import string
import contractions
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [3]:
pd.set_option('display.max_colwidth', None)

## EDA

In [4]:
train_data = pd.read_csv('../raw_data/train.csv')
train_data = train_data.drop(['id', 'orig_id'], axis=1)

In [17]:
val_data = pd.read_csv('../raw_data/validation.csv')
val_data = val_data.drop(['id', 'orig_id'], axis=1)

In [18]:
test_data = pd.read_csv('../raw_data/test.csv')
test_data = test_data.drop(['id', 'orig_id'], axis=1)

In [19]:
train_data.head(3)

,article,highlights
0,"LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in ""Harry Potter and the Order of the Phoenix"" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. ""I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar,"" he told an Australian interviewer earlier this month. ""I don't think I'll be particularly extravagant. ""The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs."" At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film ""Hostel: Part II,"" currently six places below his number one movie on the UK box office chart. Details of how he'll mark his landmark birthday are under wraps. His agent and publicist had no comment on his plans. ""I'll definitely have some sort of party,"" he said in an interview. ""Hopefully none of you will be reading about it."" Radcliffe's earnings from the first five Potter films have been held in a trust fund which he has not been able to touch. Despite his growing fame and riches, the actor says he is keeping his feet firmly on the ground. ""People are always looking to say 'kid star goes off the rails,'"" he told reporters last month. ""But I try very hard not to go that way because it would be too easy for them."" His latest outing as the boy wizard in ""Harry Potter and the Order of the Phoenix"" is breaking records on both sides of the Atlantic and he will reprise the role in the last two films. Watch I-Reporter give her review of Potter's latest » . There is life beyond Potter, however. The Londoner has filmed a TV movie called ""My Boy Jack,"" about author Rudyard Kipling and his son, due for release later this year. He will also appear in ""December Boys,"" an Australian film about four boys who escape an orphanage. Earlier this year, he made his stage debut playing a tortured teenager in Peter Shaffer's ""Equus."" Meanwhile, he is braced for even closer media scrutiny now that he's legally an adult: ""I just think I'm going to be more sort of fair game,"" he told Reuters. E-mail to a friend . Copyright 2007 Reuters. All rights reserved.This material may not be published, broadcast, rewritten, or redistributed.",Harry Potter star Daniel Radcliffe gets £20M fortune as he turns 18 Monday .\nYoung actor says he has no plans to fritter his cash away .\nRadcliffe's earnings from first five Potter films have been held in trust fund .
1,"Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the ""forgotten floor,"" where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the ""forgotten floor."" Here, inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually ""avoidable felonies."" He says the arrests often result from confrontations with police. Mentally ill people often won't do what they're told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less likely to follow directions, according to Leifman. So, they end up on the ninth floor severely mentally disturbed, but not getting any real help because they're in jail. We toured the jail with Leifman. He is well

In [20]:
print(f"train_data.shape: {train_data.shape}")

train_data.shape: (284015, 2)


In [21]:
train_data.isna().sum()

article       0
highlights    0
dtype: int64

In [22]:
val_data.isna().sum()

article       0
highlights    0
dtype: int64

In [23]:
test_data.isna().sum()

article       0
highlights    0
dtype: int64

### Dealing with duplicates

In [24]:
train_data.duplicated(subset=['article', 'highlights']).sum()

0

In [25]:
train_data = train_data.drop_duplicates(subset=['article', 'highlights'])
train_data.shape

(284015, 2)

In [26]:
val_data.duplicated(subset=['article', 'highlights']).sum()

0

In [27]:
test_data.duplicated(subset=['article', 'highlights']).sum()

2

In [28]:
test_data = test_data.drop_duplicates(subset=['article', 'highlights'])
test_data.shape

(11488, 2)

### Cleaning data

In [29]:
def preprocessing(sentence, remove_stopwords=True):
    '''
    Preprocessing text: lower case, 
                        deleting punctuation, 
                        replacing contructions with equivalent,
                        deleting stop words
    '''

    # Lowercasing
    sentence = sentence.lower()
   
    # Removing numbers
    #sentence = ''.join(char for char in sentence if not char.isdigit())
 
    # Removing punctuation
    sentence = re.sub(r'https?:\/\/.*[\r\n]*', '', sentence, flags=re.MULTILINE)
    sentence = re.sub(r'\n', ' ', sentence)
    sentence = re.sub(r'\<a href', ' ', sentence)
    sentence = re.sub(r'&amp;', '', sentence) 
    sentence = re.sub(r'[~^_`"\-;%(){|}\+&<=>\*%.,!\?:#$@\[\]/]', ' ', sentence)
    sentence = re.sub(r'<br />', ' ', sentence)
    
    expanded = []   
    for word in sentence.split():
      # using contractions.fix to expand the shortened words
      expanded.append(contractions.fix(word, slang=False))
        
    expanded_sentence = ' '.join(expanded)
    sentence = re.sub(r'\'', ' ', expanded_sentence)
    
    # Removing whitespaces
    sentence = sentence.strip()
            
    tokenized_sentence = word_tokenize(sentence) ## tokenizing 
    
    if remove_stopwords:
        stop_words = set(stopwords.words('english')) ## defining stopwords
        tokenized_sentence = [w for w in tokenized_sentence 
                                  if not w in stop_words] ## remove stopwords
    
    return tokenized_sentence
    


In [32]:
clean_articles = []
for summary in train_data['article'][:4]:
    clean_articles.append(preprocessing(summary, remove_stopwords=True))
print('Cleaning Articles Complete')


Cleaning Articles Complete


/var/folders/xt/8t3t9kmn787_rthnt36ggbyc0000gn/T/ipykernel_2546/2727114845.py:2: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  for summary in train_data['article'][:4]:


In [33]:
clean_articles[1]

['editor',
 'note',
 'behind',
 'scenes',
 'series',
 'cnn',
 'correspondents',
 'share',
 'experiences',
 'covering',
 'news',
 'analyze',
 'stories',
 'behind',
 'events',
 'soledad',
 'brien',
 'takes',
 'users',
 'inside',
 'jail',
 'many',
 'inmates',
 'mentally',
 'ill',
 'inmate',
 'housed',
 'forgotten',
 'floor',
 'many',
 'mentally',
 'ill',
 'inmates',
 'housed',
 'miami',
 'trial',
 'miami',
 'florida',
 'cnn',
 'ninth',
 'floor',
 'miami',
 'dade',
 'pretrial',
 'detention',
 'facility',
 'dubbed',
 'forgotten',
 'floor',
 'inmates',
 'severe',
 'mental',
 'illnesses',
 'incarcerated',
 'ready',
 'appear',
 'court',
 'often',
 'face',
 'drug',
 'charges',
 'charges',
 'assaulting',
 'officer',
 'charges',
 'judge',
 'steven',
 'leifman',
 'says',
 'usually',
 'avoidable',
 'felonies',
 'says',
 'arrests',
 'often',
 'result',
 'confrontations',
 'police',
 'mentally',
 'ill',
 'people',
 'often',
 'told',
 'police',
 'arrive',
 'scene',
 'confrontation',
 'seems',
 'exacer

In [34]:
clean_summaries = []
for summary in train_data['highlights'][:4]:
    clean_summaries.append(preprocessing(summary, remove_stopwords=False))
print('Cleaning Summaries Complete')


Cleaning Summaries Complete


/var/folders/xt/8t3t9kmn787_rthnt36ggbyc0000gn/T/ipykernel_2546/1731836068.py:2: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  for summary in train_data['highlights'][:4]:


In [36]:
clean_summaries[1]

['mentally',
 'ill',
 'inmates',
 'in',
 'miami',
 'are',
 'housed',
 'on',
 'the',
 'forgotten',
 'floor',
 'judge',
 'steven',
 'leifman',
 'says',
 'most',
 'are',
 'there',
 'as',
 'a',
 'result',
 'of',
 'avoidable',
 'felonies',
 'while',
 'cnn',
 'tours',
 'facility',
 'patient',
 'shouts',
 'i',
 'am',
 'the',
 'son',
 'of',
 'the',
 'president',
 'leifman',
 'says',
 'the',
 'system',
 'is',
 'unjust',
 'and',
 'he',
 'is',
 'fighting',
 'for',
 'change']